In [1]:
from elasticsearch import Elasticsearch
import json
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
# Elasticsearch 객체 생성 : https 환경
es = Elasticsearch('https://211.189.131.68:9200', http_auth=('elastic', 'yFD5IKpwnnDXfIfFCwGG'), use_ssl=True, verify_certs=False)
es.info()

{'name': 'node1',
 'cluster_name': 'my-application',
 'cluster_uuid': 'hZOvgYvjSamfiBowmk6BBQ',
 'version': {'number': '7.9.2',
  'build_flavor': 'default',
  'build_type': 'rpm',
  'build_hash': 'd34da0ea4a966c4e49417f2da2f244e3e97b4e6e',
  'build_date': '2020-09-23T00:45:33.626720Z',
  'build_snapshot': False,
  'lucene_version': '8.6.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

### 콩 생육정보

In [6]:
# time check
start_time = time.time()

doc_count = 0
index = "bean"
resp = es.search(
    index = index,
    body = {
            "size": 100,
            "query": {
                "match_all": {}
                },
            "sort": [
                {
                  "measure_time": {
                    "order": "asc"
                  }
                }
              ]
            },
    scroll = '1s' 
)

old_scroll_id = resp['_scroll_id']


res = {}
result = []


# 처음 출력된 결과 저장
for doc in resp['hits']['hits']:
    res['leaf_length'] = doc['_source']['leaf_length']
    res['tiller_count'] = doc['_source']['tiller_count']
    res['pot_count'] = doc['_source']['pod_count']
    res['measure_position'] = doc['_source']['measure_position']
    res['measure_time'] = doc['_source']['measure_time']
    result.append(res)
    

# SCROLL API를 통해 나온 결과 저장
while len(resp['hits']['hits']):
    resp = es.scroll(
        scroll_id = old_scroll_id,
        scroll = '1m' # length of time to keep search context
    )

    for doc in resp['hits']['hits']:
        res['leaf_length'] = doc['_source']['leaf_length']
        res['tiller_count'] = doc['_source']['tiller_count']
        res['pot_count'] = doc['_source']['pod_count']
        res['measure_position'] = doc['_source']['measure_position']
        res['measure_time'] = doc['_source']['measure_time']
        result.append(res)
        
    print(len(result))      

print ("TOTAL TIME:", time.time() - start_time, "seconds.")

200
300
400
500
520
520
TOTAL TIME: 0.038832664489746094 seconds.


In [7]:
# 데이터프레임화
bean_df = pd.DataFrame(result)
bean_df.head()

,leaf_length,tiller_count,pot_count,measure_position,measure_time
0,95.0,10,20,POINT(127.82766158352298 36.88688556298606),2020-07-25T03:00:00.000Z
1,95.0,10,20,POINT(127.82766158352298 36.88688556298606),2020-07-25T03:00:00.000Z
2,95.0,10,20,POINT(127.82766158352298 36.88688556298606),2020-07-25T03:00:00.000Z
3,95.0,10,20,POINT(127.82766158352298 36.88688556298606),2020-07-25T03:00:00.000Z
4,95.0,10,20,POINT(127.82766158352298 36.88688556298606),2020-07-25T03:00:00.000Z


In [8]:
bean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   leaf_length       520 non-null    float64
 1   tiller_count      520 non-null    int64  
 2   pot_count         520 non-null    int64  
 3   measure_position  520 non-null    object 
 4   measure_time      520 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 20.4+ KB


### 기상관측

In [51]:
# time check
start_time = time.time()

doc_count = 0
index = "weather"
resp = es.search(
    index = index,
    body = {
            "size": 100,
            "query": {
                "match_all": {}
                },
            "sort": [
                {
                  "measure_time": {
                    "order": "asc"
                  }
                }
              ]
            },
    scroll = '1s' 
)

old_scroll_id = resp['_scroll_id']


res = {}
result = []


# 처음 출력된 결과 저장
for doc in resp['hits']['hits']:
    res['temperature'] = doc['_source']['temperature']
    res['humidity'] = doc['_source']['humidity']
    res['wind_velocity'] = doc['_source']['wind_velocity']
    res['wind_direction'] = doc['_source']['wind_direction']
    res['daylight_hour'] = doc['_source']['daylight_hour']
    res['rainfall'] = doc['_source']['rainfall']
    res['rainfall_detection'] = doc['_source']['rainfall_detection']
    res['measure_time'] = doc['_source']['measure_time']
    res['measure_position'] = doc['_source']['measure_position']
    result.append(res)

# SCROLL API를 통해 나온 결과 저장
while len(resp['hits']['hits']):
    resp = es.scroll(
        scroll_id = old_scroll_id,
        scroll = '1m' # length of time to keep search context
    )

    for doc in resp['hits']['hits']:
        res['temperature'] = doc['_source']['temperature']
        res['humidity'] = doc['_source']['humidity']
        res['wind_velocity'] = doc['_source']['wind_velocity']
        res['wind_direction'] = doc['_source']['wind_direction']
        res['daylight_hour'] = doc['_source']['daylight_hour']
        res['rainfall'] = doc['_source']['rainfall']
        res['rainfall_detection'] = doc['_source']['rainfall_detection']
        res['measure_time'] = doc['_source']['measure_time']
        res['measure_position'] = doc['_source']['measure_position']
        result.append(res)
        
    print(len(result))      

print ("TOTAL TIME:", time.time() - start_time, "seconds.")

139
139
TOTAL TIME: 0.013741254806518555 seconds.


In [53]:
resp

{'_scroll_id': 'FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFDZnZ3JRSGtCNFVjd3FmWldoNjc5AAAAAAAABpoWblRocm9mbEtRRTJ3V2tQNEE1bTh3UQ==',
 'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 139, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [14]:
# 데이터프레임화
weather_df = pd.DataFrame(result)
weather_df.head()

,temperature,humidity,wind_velocity,wind_direction,daylight_hour,rainfall,rainfall_detection,measure_time,measure_position
0,26.12,76.63,0.05,None,5.0,1.0,1,2020-07-25T03:00:00.000Z,POINT(127.8196775 36.889491)
1,26.12,76.63,0.05,None,5.0,1.0,1,2020-07-25T03:00:00.000Z,POINT(127.8196775 36.889491)
2,26.12,76.63,0.05,None,5.0,1.0,1,2020-07-25T03:00:00.000Z,POINT(127.8196775 36.889491)
3,26.12,76.63,0.05,None,5.0,1.0,1,2020-07-25T03:00:00.000Z,POINT(127.8196775 36.889491)
4,26.12,76.63,0.05,None,5.0,1.0,1,2020-07-25T03:00:00.000Z,POINT(127.8196775 36.889491)


In [15]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   temperature         139 non-null    float64
 1   humidity            139 non-null    float64
 2   wind_velocity       139 non-null    float64
 3   wind_direction      0 non-null      object 
 4   daylight_hour       139 non-null    float64
 5   rainfall            139 non-null    float64
 6   rainfall_detection  139 non-null    int64  
 7   measure_time        139 non-null    object 
 8   measure_position    139 non-null    object 
dtypes: float64(5), int64(1), object(3)
memory usage: 9.9+ KB


### 토양센서

In [24]:
# time check
start_time = time.time()

doc_count = 0
index = "soil_sensor"
resp = es.search(
    index = index,
    body = {
            "size": 100,
            "query": {
                "match_all": {}
                },
            "sort": [
                {
                  "measure_time": {
                    "order": "asc"
                  }
                }
              ]
            },
    scroll = '1s' 
)

old_scroll_id = resp['_scroll_id']


res = {}
result = []


# 처음 출력된 결과 저장
for doc in resp['hits']['hits']:
    res['temperature'] = doc['_source']['temperature']
    res['moisture'] = doc['_source']['moisture']
    res['ec'] = doc['_source']['ec']
    res['ph'] = doc['_source']['ph']
    res['measure_time'] = doc['_source']['measure_time']
    res['measure_position'] = doc['_source']['measure_position']
    result.append(res)

# SCROLL API를 통해 나온 결과 저장
while len(resp['hits']['hits']):
    resp = es.scroll(
        scroll_id = old_scroll_id,
        scroll = '1m' # length of time to keep search context
    )

    for doc in resp['hits']['hits']:
        res['temperature'] = doc['_source']['temperature']
        res['moisture'] = doc['_source']['moisture']
        res['ec'] = doc['_source']['ec']
        res['ph'] = doc['_source']['ph']
        res['measure_time'] = doc['_source']['measure_time']
        res['measure_position'] = doc['_source']['measure_position']
        result.append(res)
        
    print(len(result))      

print ("TOTAL TIME:", time.time() - start_time, "seconds.")

200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18500
18

In [18]:
df = pd.DataFrame(result)
df.head()

,temperature,moisture,ec,ph,measure_time,measure_position
0,22.88,23.85,0.105155,5.554089,2020-07-25T03:00:00.000Z,POINT(127.82687041505895 36.889999295269035)
1,22.88,23.85,0.105155,5.554089,2020-07-25T03:00:00.000Z,POINT(127.82687041505895 36.889999295269035)
2,22.88,23.85,0.105155,5.554089,2020-07-25T03:00:00.000Z,POINT(127.82687041505895 36.889999295269035)
3,22.88,23.85,0.105155,5.554089,2020-07-25T03:00:00.000Z,POINT(127.82687041505895 36.889999295269035)
4,22.88,23.85,0.105155,5.554089,2020-07-25T03:00:00.000Z,POINT(127.82687041505895 36.889999295269035)


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20850 entries, 0 to 20849
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   temperature       20850 non-null  float64
 1   moisture          20850 non-null  float64
 2   ec                20850 non-null  float64
 3   ph                20850 non-null  float64
 4   measure_time      20850 non-null  object 
 5   measure_position  20850 non-null  object 
dtypes: float64(4), object(2)
memory usage: 977.5+ KB


### parcel 필지정보

In [100]:
# time check
start_time = time.time()

doc_count = 0
index = "parcel"
resp = es.search(
    index = index,
    body = {
            "size": 100,
            "query": {
                "match_all": {}
                }
            },
    scroll = '1s' 
)

old_scroll_id = resp['_scroll_id']


res = {}
result = []


# 처음 출력된 결과 저장
for doc in resp['hits']['hits']:
    res['pnu'] = doc['_source']['pnu']
    res['pnu_sub'] = doc['_source']['pnu_sub']
    res['addr'] = doc['_source']['addr']
    res['addr_sido'] = doc['_source']['addr_sido']
    res['addr_sigoon'] = doc['_source']['addr_sigoon']
    res['addr_dong'] = doc['_source']['addr_dong']
    res['jimok'] = doc['_source']['jimok']
    res['addr_li'] = doc['_source']['addr_li']
    res['geom'] = doc['_source']['geom']
    res['id'] = doc['_source']['id']
    result.append(res)

# SCROLL API를 통해 나온 결과 저장
while len(resp['hits']['hits']):
    resp = es.scroll(
        scroll_id = old_scroll_id,
        scroll = '1s' # length of time to keep search context
    )

    for doc in resp['hits']['hits']:
        res['pnu'] = doc['_source']['pnu']
        res['pnu_sub'] = doc['_source']['pnu_sub']
        res['addr'] = doc['_source']['addr']
        res['addr_sido'] = doc['_source']['addr_sido']
        res['addr_sigoon'] = doc['_source']['addr_sigoon']
        res['addr_dong'] = doc['_source']['addr_dong']
        res['jimok'] = doc['_source']['jimok']
        res['addr_li'] = doc['_source']['addr_li']
        res['geom'] = doc['_source']['geom']
        res['id'] = doc['_source']['id']
        result.append(res)
        
    print(len(result))      

print ("TOTAL TIME:", time.time() - start_time, "seconds.")

200
300
400
402
402
TOTAL TIME: 0.0328066349029541 seconds.


In [102]:
result

[{'pnu': '4376041023111170000',
  'pnu_sub': 0,
  'addr': '불정면 탑촌리 1117',
  'addr_sido': '충청북도',
  'addr_sigoon': '괴산군',
  'addr_dong': '불정면',
  'jimok': '답',
  'addr_li': '탑촌리',
  'geom': 'MULTIPOLYGON(((127.82161209883492 36.89309165096047,127.82260306200894 36.892718614179,127.82250785725205 36.89255464726567,127.82151065725414 36.89293178109148,127.82161209883492 36.89309165096047)))',
  'id': 60},
 {'pnu': '4376041023111170000',
  'pnu_sub': 0,
  'addr': '불정면 탑촌리 1117',
  'addr_sido': '충청북도',
  'addr_sigoon': '괴산군',
  'addr_dong': '불정면',
  'jimok': '답',
  'addr_li': '탑촌리',
  'geom': 'MULTIPOLYGON(((127.82161209883492 36.89309165096047,127.82260306200894 36.892718614179,127.82250785725205 36.89255464726567,127.82151065725414 36.89293178109148,127.82161209883492 36.89309165096047)))',
  'id': 60},
 {'pnu': '4376041023111170000',
  'pnu_sub': 0,
  'addr': '불정면 탑촌리 1117',
  'addr_sido': '충청북도',
  'addr_sigoon': '괴산군',
  'addr_dong': '불정면',
  'jimok': '답',
  'addr_li': '탑촌리',
  'geom':

In [101]:
# 데이터프레임화
parcel_df = pd.DataFrame(result)
parcel_df.head()

,pnu,pnu_sub,addr,addr_sido,addr_sigoon,addr_dong,jimok,addr_li,geom,id
0,4376041023111170000,0,불정면 탑촌리 1117,충청북도,괴산군,불정면,답,탑촌리,MULTIPOLYGON(((127.82161209883492 36.893091650...,60
1,4376041023111170000,0,불정면 탑촌리 1117,충청북도,괴산군,불정면,답,탑촌리,MULTIPOLYGON(((127.82161209883492 36.893091650...,60
2,4376041023111170000,0,불정면 탑촌리 1117,충청북도,괴산군,불정면,답,탑촌리,MULTIPOLYGON(((127.82161209883492 36.893091650...,60
3,4376041023111170000,0,불정면 탑촌리 1117,충청북도,괴산군,불정면,답,탑촌리,MULTIPOLYGON(((127.82161209883492 36.893091650...,60
4,4376041023111170000,0,불정면 탑촌리 1117,충청북도,괴산군,불정면,답,탑촌리,MULTIPOLYGON(((127.82161209883492 36.893091650...,60


In [31]:
parcel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   pnu          402 non-null    object
 1   pnu_sub      402 non-null    int64 
 2   addr         402 non-null    object
 3   addr_sido    402 non-null    object
 4   addr_sigoon  402 non-null    object
 5   addr_dong    402 non-null    object
 6   jimok        402 non-null    object
 7   addr_li      402 non-null    object
 8   geom         402 non-null    object
 9   id           402 non-null    int64 
dtypes: int64(2), object(8)
memory usage: 31.5+ KB


### parcel_crop 필지별 작물

In [64]:
# time check
start_time = time.time()

doc_count = 0
index = "weather"
resp = es.search(
    index = index,
    body = {
            "size": 100,
            "query": {
                "match_all": {}
                }
            },
    scroll = '30s' 
)

sid = resp['_scroll_id']
fetched = len(resp['hits']['hits'])

nums = []

    
res = {}
result = []


for i in range(fetched):
    nums.append(int(resp['hits']['hits'][i]['_source']['temperature']))
    
while(fetched>0):
    resp = es.scroll(scroll_id=sid, scroll='30s')
    fetched = len(resp['hits']['hits'])
    for i in range(fetched):
        nums.append(int(resp['hits']['hits'][i]['_source']['temperature']))
print(nums)


[23, 22, 22, 25, 28, 24, 28, 26, 10, 12, 15, 24, 23, 24, 7, 24, 29, 25, 25, 23, 21, 31, 22, 31, 16, 21, 28, 26, 28, 10, 16, 29, 13, 14, 7, 13, 7, 19, 15, 20, 12, 15, 14, 16, 18, 14, 17, 14, 15, 17, 15, 22, 26, 24, 20, 10, 24, 16, 11, 27, 25, 26, 29, 26, 8, 13, 21, 24, 15, 8, 26, 23, 26, 13, 24, 26, 18, 22, 14, 24, 16, 15, 13, 19, 22, 20, 29, 31, 22, 29, 26, 29, 14, 16, 19, 21, 18, 25, 21, 21, 13, 23, 23, 14, 12, 17, 15, 19, 22, 8, 20, 13, 15, 21, 16, 24, 20, 21, 18, 22, 27, 27, 21, 27, 27, 29, 28, 29, 21, 15, 21, 25, 27, 31, 26, 27, 24, 26, 26]


In [146]:
# time check
start_time = time.time()

doc_count = 0
index = "parcel_crop"
resp = es.search(
    index = index,
    body = {
            "size": 100,
            "query": {
                "match_all": {}
                }
            },
    scroll = '1s' 
)

sid = resp['_scroll_id']
fetched = len(resp['hits']['hits'])

   
res = {}
result = []

# # 처음 출력된 결과 저장
# for i in range(fetched):
#     for doc in resp['hits']['hits']:
#         res['pnu'] = doc[i]['_source']['pnu']
#         res['pnu_sub'] = doc[i]['_source']['pnu_sub']
#         res['crop'] = doc[i]['_source']['crop']
#         res['variety'] = doc[[i]['_source']['variety']
#     result.append(res)


# for doc in resp['hits']['hits']:
#     res['pnu'] = doc['_source']['pnu']
#     res['pnu_sub'] = doc['_source']['pnu_sub']
#     res['crop'] = doc['_source']['crop']
#     res['variety'] = doc['_source']['variety']
#     result.append(res)

# # SCROLL API를 통해 나온 결과 저장
# while len(resp['hits']['hits']):
#     resp = es.scroll(
#         scroll_id = sid,
#         scroll = '1s' # length of time to keep search context
#     )

#     for doc in resp['hits']['hits']:
#         res['pnu'] = doc['_source']['pnu']
#         res['pnu_sub'] = doc['_source']['pnu_sub']
#         res['crop'] = doc['_source']['crop']
#         res['variety'] = doc['_source']['variety']
#         result.append(res)
        
#     print(len(result))      

# print ("TOTAL TIME:", time.time() - start_time, "seconds.")

In [143]:
result

[{'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},

In [147]:
resp

{'_scroll_id': 'FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFFhBaGlRSGtCNFVjd3FmWldLYlM1AAAAAAAADAwWblRocm9mbEtRRTJ3V2tQNEE1bTh3UQ==',
 'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 193, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'parcel_crop',
    '_type': '_doc',
    '_id': 'ubfF_XgBczqWb-mafMVD',
    '_score': 1.0,
    '_source': {'@version': '1',
     'crop': '콩',
     'pnu_sub': 0,
     'variety': '대원콩',
     'pnu': '4376041023111380000',
     '@timestamp': '2021-04-23T08:08:00.249Z',
     'type': 'parcel_crop'}},
   {'_index': 'parcel_crop',
    '_type': '_doc',
    '_id': 'urfF_XgBczqWb-mafMVD',
    '_score': 1.0,
    '_source': {'@version': '1',
     'crop': '콩',
     'pnu_sub': 0,
     'variety': '대원콩',
     'pnu': '4376041023111390000',
     '@timestamp': '2021-04-23T08:08:00.250Z',
     'type': 'parcel_crop'}},
   {'_index': 'parcel_crop',
    '_type': '_doc',
    '_id'

In [117]:
fetched

100

In [140]:
test ={}
test_list =[]


for doc in resp['hits']['hits']:
    test['pnu'] = resp['hits']['hits'][13]['_source']['pnu']
    test['pnu'] = resp['hits']['hits'][13]['_source']['pnu']
    test['pnu_sub'] = resp['hits']['hits'][13]['_source']['pnu_sub']
    test['crop'] = resp['hits']['hits'][13]['_source']['crop']
    test['variety'] = resp['hits']['hits'][13]['_source']['variety']
    test_list.append(test)

test_list

[]

In [148]:
fetched = len(resp['hits']['hits'])

for doc in resp['hits']['hits']:
    for i in range(fetched):
        test['pnu'] = int(doc[i]['_source']['pnu'])
        test['pnu'] = doc[i]['_source']['pnu']
        test['pnu_sub'] = doc[i]['_source']['pnu_sub']
        test['crop'] = doc[i]['_source']['crop']
        test['variety'] = doc[i]['_source']['variety']
        test_list.append(test)

test_list

KeyError: 0

In [114]:
test_df = pd.DataFrame(test)
test_df

ValueError: If using all scalar values, you must pass an index

In [82]:
fetched

100

In [62]:
nums_df = pd.DataFrame(nums)
nums_df.head()

,0
0,23
1,22
2,22
3,25
4,28


In [60]:
resp

{'_scroll_id': 'FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFE9BZ3VRSGtCNFVjd3FmWldkNjlPAAAAAAAABugWblRocm9mbEtRRTJ3V2tQNEE1bTh3UQ==',
 'took': 1,
 'timed_out': False,
 'terminated_early': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 139, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': []}}

In [42]:
index = "parcel_crop"
resp = es.search(
    index = index,
    body = {
            "size": 193,
            "query": {
                "match_all": {}
                }
            },
    scroll = '1s' 
)
resp

{'_scroll_id': 'FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFDVRZ2hRSGtCNFVjd3FmWldoSzB5AAAAAAAABZUWblRocm9mbEtRRTJ3V2tQNEE1bTh3UQ==',
 'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 193, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'parcel_crop',
    '_type': '_doc',
    '_id': 'ubfF_XgBczqWb-mafMVD',
    '_score': 1.0,
    '_source': {'@version': '1',
     'crop': '콩',
     'pnu_sub': 0,
     'variety': '대원콩',
     'pnu': '4376041023111380000',
     '@timestamp': '2021-04-23T08:08:00.249Z',
     'type': 'parcel_crop'}},
   {'_index': 'parcel_crop',
    '_type': '_doc',
    '_id': 'urfF_XgBczqWb-mafMVD',
    '_score': 1.0,
    '_source': {'@version': '1',
     'crop': '콩',
     'pnu_sub': 0,
     'variety': '대원콩',
     'pnu': '4376041023111390000',
     '@timestamp': '2021-04-23T08:08:00.250Z',
     'type': 'parcel_crop'}},
   {'_index': 'parcel_crop',
    '_type': '_doc',
    '_id'

In [39]:
result

[{'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},
 {'pnu': '4376041023111060000', 'pnu_sub': 0, 'crop': '콩', 'variety': '선풍'},

In [37]:
# 데이터프레임화
parcelcrop_df = pd.DataFrame(result)
parcelcrop_df.head()

,pnu,pnu_sub,crop,variety
0,4376041023111060000,0,콩,선풍
1,4376041023111060000,0,콩,선풍
2,4376041023111060000,0,콩,선풍
3,4376041023111060000,0,콩,선풍
4,4376041023111060000,0,콩,선풍


In [34]:
parcelcrop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   pnu      193 non-null    object
 1   pnu_sub  193 non-null    int64 
 2   crop     193 non-null    object
 3   variety  193 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.2+ KB


## EDA

In [35]:
# bean 통계
bean_df.describe()

,leaf_length,tiller_count,pot_count
count,520.0,520.0,520.0
mean,95.0,10.0,20.0
std,0.0,0.0,0.0
min,95.0,10.0,20.0
25%,95.0,10.0,20.0
50%,95.0,10.0,20.0
75%,95.0,10.0,20.0
max,95.0,10.0,20.0
